# Cost Analysis

## 1. Import Library

In [1]:
import pandas as pd
import numpy as np
import pyodbc # SQL Connection
import sqlCredentials as sql

## 2. Connect to Databases

In [2]:
#PROCUREMENTDB

proc_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=PROCUREMENTDB;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

#BookXCenterProduction
prod_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=BookXCenterProduction;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

## 3. Define Clean up tool

In [3]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

## 4. Define Tables

In [4]:
# Dictionary
pub_dict = """
SELECT *
FROM dbo.PublisherDictionary
"""
# LP All currencies
all_lp = """
SELECT
	[Isbn]
	,[Currency]
	,[Price]
FROM
	[Isbn].[ListPrice]
"""
# Bibliography
bilblo_sql = """
SELECT Isbn
    , Title
    , Publisher
    , Author
FROM 
    Isbn.Bibliography
"""
# Echange Rate
exchange_rate = """
SELECT 
    name,
    rate
FROM dbo.XChange
"""
#Supplier Mega List
megalist_sql = """
SELECT 
    [ISBN]
    ,[Supplier]
    ,[Publisher]
    ,[Currency]
    ,[ListPrice]
    ,[Discount]
    ,[CostUnitPrice]
    ,[MaxQtyPerOrder]
    ,[ShipmentOrigin]
    ,[UnitShippingCost]
  FROM 
    [Process].[SupplierMegaList]
"""

#Import supplier Excel
#Import short_discount
#Import restrictions

### 4.1 Import Dictionary and clean it

In [5]:
pub_dict = pd.read_sql(pub_dict, proc_db)
pub_dict.head()

,PublisherLong,PublisherShort
0,Benning,Benning
1,"Abrams, Inc.",HACHETTE LIVE
2,"Addison-Wesley Longman, Incorporated",PEARSON
3,"Addison-Wesley Longman, Limited",PEARSON
4,"Allyn & Bacon, Incorporated",PEARSON


In [6]:
pub_dict.columns = map(str.lower, pub_dict.columns)
pub_dict = pub_dict.fillna('N/A')
for col in list(pub_dict.columns):
    pub_dict[col] = pub_dict.apply(lambda x: clean_up(x[col]), axis =1)

pub_dict.head()

,publisherlong,publishershort
0,BENNING,BENNING
1,"ABRAMS, INC.",HACHETTE LIVE
2,"ADDISON-WESLEY LONGMAN, INCORPORATED",PEARSON
3,"ADDISON-WESLEY LONGMAN, LIMITED",PEARSON
4,"ALLYN & BACON, INCORPORATED",PEARSON


### 4.2 Import Bibliography and clean it

In [7]:
# IMport and read biblio
biblio = pd.read_sql(bilblo_sql, prod_db)
biblio.head()

,Isbn,Title,Publisher,Author
0,9780000002402,None,LWW,None
1,9780000002679,None,LWW,None
2,9780000003102,None,LWW,None
3,9780000003201,None,LWW,None
4,9780000006493,None,LWW,None


In [8]:
biblio.columns = map(str.lower, biblio.columns)
biblio = biblio.fillna('N/A')
biblio['isbn'] = biblio['isbn'].astype(str)
for col in list(biblio.columns):
    biblio[col] = biblio.apply(lambda x: clean_up(x[col]), axis =1)
biblio.head()

,isbn,title,publisher,author
0,9780000002402,N/A,LWW,N/A
1,9780000002679,N/A,LWW,N/A
2,9780000003102,N/A,LWW,N/A
3,9780000003201,N/A,LWW,N/A
4,9780000006493,N/A,LWW,N/A


Now we Merge bibliography with the dictionary

In [9]:
biblio2 = pd.merge(biblio, pub_dict, how='left', left_on = 'publisher', right_on = 'publisherlong' )
biblio2.drop(columns = ['publisherlong'], inplace = True)
biblio2.rename(columns={'publishershort': 'pub'}, inplace = True)
biblio2.head()

,isbn,title,publisher,author,pub
0,9780000002402,N/A,LWW,N/A,LWW
1,9780000002679,N/A,LWW,N/A,LWW
2,9780000003102,N/A,LWW,N/A,LWW
3,9780000003201,N/A,LWW,N/A,LWW
4,9780000006493,N/A,LWW,N/A,LWW


### 4.3 Import SupplierMegaList and clean it

In [10]:
megalist = pd.read_sql(megalist_sql,proc_db)
megalist.head()

,ISBN,Supplier,Publisher,Currency,ListPrice,Discount,CostUnitPrice,MaxQtyPerOrder,ShipmentOrigin,UnitShippingCost
0,9780262530873,BILLSON,UNIVERSITY PRESS,GBP,4.99,0.38,3.09,1000,UK,3.47
1,9780262530989,BILLSON,UNIVERSITY PRESS,GBP,25.00,0.38,15.50,1000,UK,3.47
2,9781681883052,Zookal,HARPER COLLINS,AUD,0.00,0.00,24.50,150,AUS,3.47
3,9780262531092,BILLSON,UNIVERSITY PRESS,GBP,37.95,0.38,23.53,1000,UK,3.47
4,9781681883212,Zookal,HARPER COLLINS,AUD,0.00,0.00,21.00,150,AUS,3.47


In [11]:
megalist.columns = map(str.lower, megalist.columns)
megalist = megalist.fillna('N/A')
megalist['isbn'] = megalist['isbn'].astype(str)
for col in ['isbn', 'supplier', 'publisher', 'currency', 'shipmentorigin']:
    megalist[col] = megalist.apply(lambda x: clean_up(x[col]), axis =1)
megalist.head()

,isbn,supplier,publisher,currency,listprice,discount,costunitprice,maxqtyperorder,shipmentorigin,unitshippingcost
0,9780262530873,BILLSON,UNIVERSITY PRESS,GBP,4.99,0.38,3.09,1000,UK,3.47
1,9780262530989,BILLSON,UNIVERSITY PRESS,GBP,25.00,0.38,15.50,1000,UK,3.47
2,9781681883052,ZOOKAL,HARPER COLLINS,AUD,0.00,0.00,24.50,150,AUS,3.47
3,9780262531092,BILLSON,UNIVERSITY PRESS,GBP,37.95,0.38,23.53,1000,UK,3.47
4,9781681883212,ZOOKAL,HARPER COLLINS,AUD,0.00,0.00,21.00,150,AUS,3.47


In [12]:
megalist_pub = pd.merge(megalist, biblio2, how='left', on = 'isbn' )
megalist_pub.head()

,isbn,supplier,publisher_x,currency,listprice,discount,costunitprice,maxqtyperorder,shipmentorigin,unitshippingcost,title,publisher_y,author,pub
0,9780262530873,BILLSON,UNIVERSITY PRESS,GBP,4.99,0.38,3.09,1000,UK,3.47,N/A,UNIVERSITY PRESS,N/A,UNIVERSITY PRESS
1,9780262530989,BILLSON,UNIVERSITY PRESS,GBP,25.00,0.38,15.50,1000,UK,3.47,N/A,N/A,N/A,N/A
2,9781681883052,ZOOKAL,HARPER COLLINS,AUD,0.00,0.00,24.50,150,AUS,3.47,N/A,N/A,N/A,N/A
3,9780262531092,BILLSON,UNIVERSITY PRESS,GBP,37.95,0.38,23.53,1000,UK,3.47,N/A,UNIVERSITY PRESS,N/A,UNIVERSITY PRESS
4,9781681883212,ZOOKAL,HARPER COLLINS,AUD,0.00,0.00,21.00,150,AUS,3.47,N/A,N/A,N/A,N/A


In [13]:
#cleaning pub name
megalist_pub['pub'] = np.where(
    megalist_pub['pub'] == 'N/A'
    , megalist_pub['publisher_x']
    , megalist_pub['pub'])
    
# centralizing price in one column
megalist_pub['price'] = np.where(megalist_pub['listprice'] == 0.00, megalist_pub['costunitprice'], megalist_pub['listprice'])

#clean column names
megalist_pub.rename(columns={'unitshippingcost': 'shipping_cost', 'maxqtyperorder': 'max_qty', 'shipmentorigin': 'origin'}, inplace = True)

#Arrange columns and drop unnecessary columns

megalist_pub = megalist_pub[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost', 'max_qty', 'origin']]

megalist_pub.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,max_qty,origin
0,9780262530873,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,4.99,0.38,3.47,1000,UK
1,9780262530989,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,25.00,0.38,3.47,1000,UK
2,9781681883052,N/A,HARPER COLLINS,N/A,ZOOKAL,AUD,24.50,0.00,3.47,150,AUS
3,9780262531092,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,37.95,0.38,3.47,1000,UK
4,9781681883212,N/A,HARPER COLLINS,N/A,ZOOKAL,AUD,21.00,0.00,3.47,150,AUS


### 4.3 Import all_lp and clean it

In [14]:
all_lp = pd.read_sql(all_lp, prod_db)
all_lp.head()

,Isbn,Currency,Price
0,9780131392632,USD,245.20
1,9780131392632,GBP,152.99
2,9780131392656,CND,53.90
3,9780131392656,USD,49.00
4,9780131392656,GBP,30.99


In [15]:
all_lp.columns = map(str.lower, all_lp.columns)
all_lp = all_lp.fillna('N/A')
all_lp['isbn'] = all_lp['isbn'].astype(str)
for col in ['isbn','currency']:
    all_lp[col] = all_lp.apply(lambda x: clean_up(x[col]), axis =1)
all_lp.head()

,isbn,currency,price
0,9780131392632,USD,245.20
1,9780131392632,GBP,152.99
2,9780131392656,CND,53.90
3,9780131392656,USD,49.00
4,9780131392656,GBP,30.99


#### 4.3.1 Get biblio with LP

In [16]:
biblio_lp = pd.merge(all_lp, biblio2, how='left', on = 'isbn' )
biblio_lp.drop(columns = ['pub'], inplace = True)
biblio_lp.head()

,isbn,currency,price,title,publisher,author
0,9780131392632,USD,245.20,N/A,N/A,N/A
1,9780131392632,GBP,152.99,N/A,N/A,N/A
2,9780131392656,CND,53.90,N/A,N/A,N/A
3,9780131392656,USD,49.00,N/A,N/A,N/A
4,9780131392656,GBP,30.99,N/A,N/A,N/A


### 4.4 Import Sup_by_pub and clean it

In [17]:
sup_pub = pd.read_excel('../../../../Vendors/.  Vendors Details\supplier_procurement_details.xlsx',  sheet_name='pub')
sup_pub.head()

,Supplier,Publisher,Currency,Discount,Shipping Cost,max quantity,Min,ShipmentOrigin
0,COINFO,3D TOTAL,AUD,0.22,3.3,10000,0,AUS
1,COINFO,ABC CLIO,AUD,0.30,3.3,10000,0,AUS
2,COINFO,AIAA,AUD,0.30,3.3,10000,0,AUS
3,COINFO,ALIEN & UNWIN,AUD,0.30,3.3,10000,0,AUS
4,COINFO,Amer Psych Ass.,AUD,0.33,3.3,10000,0,AUS


In [18]:
sup_pub.columns = map(str.lower, sup_pub.columns)
sup_pub = sup_pub.fillna('N/A')
for col in ['supplier','publisher', 'currency', 'shipmentorigin']:
    sup_pub[col] = sup_pub.apply(lambda x: clean_up(x[col]), axis =1)
sup_pub.head()

,supplier,publisher,currency,discount,shipping cost,max quantity,min,shipmentorigin
0,COINFO,3D TOTAL,AUD,0.22,3.3,10000,0,AUS
1,COINFO,ABC CLIO,AUD,0.30,3.3,10000,0,AUS
2,COINFO,AIAA,AUD,0.30,3.3,10000,0,AUS
3,COINFO,ALIEN & UNWIN,AUD,0.30,3.3,10000,0,AUS
4,COINFO,AMER PSYCH ASS.,AUD,0.33,3.3,10000,0,AUS


#### 4.4.1 Merge Supplier by Publisher with biblio

In [19]:
sup_pub_biblio = pd.merge(biblio_lp, sup_pub, how='inner', left_on = ['currency', 'publisher'], right_on = ['currency', 'publisher'])
sup_pub_biblio.head()

,isbn,currency,price,title,publisher,author,supplier,discount,shipping cost,max quantity,min,shipmentorigin
0,9780131394124,GBP,194.49,INTRODUCTION TO RISK MANAGEMENT AND INSURANCE,PEARSON,MARK S. DORFMAN; DAVID A. CATHER,ABE,0.43,3.47,10000,0,UK
1,9780131426481,GBP,140.99,SIMPLY JAVA PROGRAMMING,PEARSON,DEITEL AND ASSOCIATES STAFF; HARVEY M. DEITEL;...,ABE,0.43,3.47,10000,0,UK
2,9780131437487,GBP,87.99,EXPERIENCING GEOMETRY,PEARSON,DAVID W. HENDERSON; DAINA TAIMINA,ABE,0.43,3.47,10000,0,UK
3,9780131439825,GBP,49.99,BASIC METHODS OF STRUCTURAL GEOLOGY,PEARSON,STEPHEN MARSHAK; GAUTUM MITRA; SCOTT WILKERSON,ABE,0.43,3.47,10000,0,UK
4,9780131447707,GBP,98.99,EXERCISES IN PHYSICAL GEOLOGY,PEARSON,W. KENNETH HAMBLIN; JAMES D. HOWARD,ABE,0.43,3.47,10000,0,UK


In [20]:

sup_pub_biblio.rename(columns={'publisher': 'pub','shipping cost': 'shipping_cost', 'max quantity': 'max_qty', 'shipmentorigin': 'origin'}, inplace = True)

sup_pub_biblio.head()

,isbn,currency,price,title,pub,author,supplier,discount,shipping_cost,max_qty,min,origin
0,9780131394124,GBP,194.49,INTRODUCTION TO RISK MANAGEMENT AND INSURANCE,PEARSON,MARK S. DORFMAN; DAVID A. CATHER,ABE,0.43,3.47,10000,0,UK
1,9780131426481,GBP,140.99,SIMPLY JAVA PROGRAMMING,PEARSON,DEITEL AND ASSOCIATES STAFF; HARVEY M. DEITEL;...,ABE,0.43,3.47,10000,0,UK
2,9780131437487,GBP,87.99,EXPERIENCING GEOMETRY,PEARSON,DAVID W. HENDERSON; DAINA TAIMINA,ABE,0.43,3.47,10000,0,UK
3,9780131439825,GBP,49.99,BASIC METHODS OF STRUCTURAL GEOLOGY,PEARSON,STEPHEN MARSHAK; GAUTUM MITRA; SCOTT WILKERSON,ABE,0.43,3.47,10000,0,UK
4,9780131447707,GBP,98.99,EXERCISES IN PHYSICAL GEOLOGY,PEARSON,W. KENNETH HAMBLIN; JAMES D. HOWARD,ABE,0.43,3.47,10000,0,UK


#### 4.5 Import short_disc and clean it

In [21]:
sp_disc = pd.read_excel("../../../../Vendors/. Short Discounted/all_short_discount_list.xlsx")
sp_disc.head()

,SUPPLIER,ISBN,DISCOUNT
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


In [22]:
sp_disc.columns = map(str.lower, sp_disc.columns)
sp_disc = sp_disc.fillna('N/A')
sp_disc['isbn'] = sp_disc['isbn'].astype(str)
for col in ['supplier', 'isbn']:
    sp_disc[col] = sp_disc.apply(lambda x: clean_up(x[col]), axis =1)
sp_disc.head()

,supplier,isbn,discount
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


#### 4.5.1 Merge sup_pub_biblio by sp_disc with biblio

In [23]:
sup_biblio_disc = pd.merge(sup_pub_biblio, sp_disc, how='left', on = ['supplier', 'isbn'])
sup_biblio_disc.head()

,isbn,currency,price,title,pub,author,supplier,discount_x,shipping_cost,max_qty,min,origin,discount_y
0,9780131394124,GBP,194.49,INTRODUCTION TO RISK MANAGEMENT AND INSURANCE,PEARSON,MARK S. DORFMAN; DAVID A. CATHER,ABE,0.43,3.47,10000,0,UK,NaN
1,9780131426481,GBP,140.99,SIMPLY JAVA PROGRAMMING,PEARSON,DEITEL AND ASSOCIATES STAFF; HARVEY M. DEITEL;...,ABE,0.43,3.47,10000,0,UK,NaN
2,9780131437487,GBP,87.99,EXPERIENCING GEOMETRY,PEARSON,DAVID W. HENDERSON; DAINA TAIMINA,ABE,0.43,3.47,10000,0,UK,NaN
3,9780131439825,GBP,49.99,BASIC METHODS OF STRUCTURAL GEOLOGY,PEARSON,STEPHEN MARSHAK; GAUTUM MITRA; SCOTT WILKERSON,ABE,0.43,3.47,10000,0,UK,NaN
4,9780131447707,GBP,98.99,EXERCISES IN PHYSICAL GEOLOGY,PEARSON,W. KENNETH HAMBLIN; JAMES D. HOWARD,ABE,0.43,3.47,10000,0,UK,NaN


In [24]:
sup_biblio_disc['discount'] = np.where(sup_biblio_disc['discount_y'].isna(), sup_biblio_disc['discount_x'],sup_biblio_disc['discount_y'])


#Drop unneccessary Columns

sup_biblio_disc.head()

,isbn,currency,price,title,pub,author,supplier,discount_x,shipping_cost,max_qty,min,origin,discount_y,discount
0,9780131394124,GBP,194.49,INTRODUCTION TO RISK MANAGEMENT AND INSURANCE,PEARSON,MARK S. DORFMAN; DAVID A. CATHER,ABE,0.43,3.47,10000,0,UK,NaN,0.43
1,9780131426481,GBP,140.99,SIMPLY JAVA PROGRAMMING,PEARSON,DEITEL AND ASSOCIATES STAFF; HARVEY M. DEITEL;...,ABE,0.43,3.47,10000,0,UK,NaN,0.43
2,9780131437487,GBP,87.99,EXPERIENCING GEOMETRY,PEARSON,DAVID W. HENDERSON; DAINA TAIMINA,ABE,0.43,3.47,10000,0,UK,NaN,0.43
3,9780131439825,GBP,49.99,BASIC METHODS OF STRUCTURAL GEOLOGY,PEARSON,STEPHEN MARSHAK; GAUTUM MITRA; SCOTT WILKERSON,ABE,0.43,3.47,10000,0,UK,NaN,0.43
4,9780131447707,GBP,98.99,EXERCISES IN PHYSICAL GEOLOGY,PEARSON,W. KENNETH HAMBLIN; JAMES D. HOWARD,ABE,0.43,3.47,10000,0,UK,NaN,0.43


In [25]:
sup_biblio_disc = sup_biblio_disc[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost', 'max_qty', 'origin']]
sup_biblio_disc.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,max_qty,origin
0,9780131394124,INTRODUCTION TO RISK MANAGEMENT AND INSURANCE,PEARSON,MARK S. DORFMAN; DAVID A. CATHER,ABE,GBP,194.49,0.43,3.47,10000,UK
1,9780131426481,SIMPLY JAVA PROGRAMMING,PEARSON,DEITEL AND ASSOCIATES STAFF; HARVEY M. DEITEL;...,ABE,GBP,140.99,0.43,3.47,10000,UK
2,9780131437487,EXPERIENCING GEOMETRY,PEARSON,DAVID W. HENDERSON; DAINA TAIMINA,ABE,GBP,87.99,0.43,3.47,10000,UK
3,9780131439825,BASIC METHODS OF STRUCTURAL GEOLOGY,PEARSON,STEPHEN MARSHAK; GAUTUM MITRA; SCOTT WILKERSON,ABE,GBP,49.99,0.43,3.47,10000,UK
4,9780131447707,EXERCISES IN PHYSICAL GEOLOGY,PEARSON,W. KENNETH HAMBLIN; JAMES D. HOWARD,ABE,GBP,98.99,0.43,3.47,10000,UK


### 4.6 Create table for WC

In [26]:
can_sup = biblio_lp[biblio_lp['currency'] == 'CND']
can_sup.columns = map(str.lower, can_sup.columns)
can_sup['supplier'] = 'WC'
can_sup['discount'] = np.where(can_sup['price']<150, 0.1, np.where((can_sup['price'] >=150) & (can_sup['price']<200),0.12,0.14))
can_sup['max_qty'] = 10000
can_sup['origin'] = 'CAN'
can_sup['shipping_cost'] = 2
can_sup.rename(columns={'publisher': 'pub'}, inplace = True)


can_sup = can_sup[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost', 'max_qty', 'origin']]

can_sup.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,max_qty,origin
2,9780131392656,N/A,N/A,N/A,WC,CND,53.90,0.10,2,10000,CAN
7,9780131394124,INTRODUCTION TO RISK MANAGEMENT AND INSURANCE,PEARSON,MARK S. DORFMAN; DAVID A. CATHER,WC,CND,211.05,0.14,2,10000,CAN
9,9780131394254,N/A,N/A,N/A,WC,CND,224.90,0.14,2,10000,CAN
12,9780131394650,N/A,N/A,N/A,WC,CND,312.45,0.14,2,10000,CAN
15,9780131394681,N/A,N/A,N/A,WC,CND,203.15,0.14,2,10000,CAN


## 5. Append SupplierMegaLis, PublisherSuppliers and WesterCampus

In [27]:
all_sup = megalist_pub.append([sup_biblio_disc, can_sup], ignore_index= True)
all_sup.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,max_qty,origin
0,9780262530873,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,4.99,0.38,3.47,1000,UK
1,9780262530989,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,25.00,0.38,3.47,1000,UK
2,9781681883052,N/A,HARPER COLLINS,N/A,ZOOKAL,AUD,24.50,0.00,3.47,150,AUS
3,9780262531092,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,37.95,0.38,3.47,1000,UK
4,9781681883212,N/A,HARPER COLLINS,N/A,ZOOKAL,AUD,21.00,0.00,3.47,150,AUS


## 6. Add Fees

In [28]:
#Invoice Fee
all_sup['invoice_fee'] = np.where(
    all_sup['supplier'].str.contains('ALEK'), 0.005,
    np.where(
        (all_sup['supplier'].str.contains('LAURENTIU')) | (all_sup['supplier'].str.contains('SENAD')),0.02,
        np.where(
        (all_sup['supplier'].str.contains('ARMANDO')) | (all_sup['supplier'].str.contains('FELIPE')),0.01,
        np.where(
            (all_sup['supplier'] == 'COINFO'), -1/11,
            0)
            )
            )
            )
# lp_fee

all_sup['lp_fee'] = np.where(
    (all_sup['supplier'].str.contains('BILLSON') & (all_sup['discount'] == 0 )), 0.03,0)

all_sup.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,max_qty,origin,invoice_fee,lp_fee
0,9780262530873,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,4.99,0.38,3.47,1000,UK,0.0,0.0
1,9780262530989,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,25.00,0.38,3.47,1000,UK,0.0,0.0
2,9781681883052,N/A,HARPER COLLINS,N/A,ZOOKAL,AUD,24.50,0.00,3.47,150,AUS,0.0,0.0
3,9780262531092,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,37.95,0.38,3.47,1000,UK,0.0,0.0
4,9781681883212,N/A,HARPER COLLINS,N/A,ZOOKAL,AUD,21.00,0.00,3.47,150,AUS,0.0,0.0


## 7. import and Merge exchage rate with all suppliers

In [29]:
exch_df = pd.read_sql(exchange_rate,proc_db)
exch_df.rename(columns={'name': 'currency'}, inplace = True)
all_sup_exc = pd.merge(all_sup, exch_df, how='left', on = 'currency')
all_sup_exc.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate
0,9780262530873,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,4.99,0.38,3.47,1000,UK,0.0,0.0,1.231680
1,9780262530989,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,25.00,0.38,3.47,1000,UK,0.0,0.0,1.231680
2,9781681883052,N/A,HARPER COLLINS,N/A,ZOOKAL,AUD,24.50,0.00,3.47,150,AUS,0.0,0.0,0.662076
3,9780262531092,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,37.95,0.38,3.47,1000,UK,0.0,0.0,1.231680
4,9781681883212,N/A,HARPER COLLINS,N/A,ZOOKAL,AUD,21.00,0.00,3.47,150,AUS,0.0,0.0,0.662076


## 8. Cost Before exchange rate and landed cost

In [30]:
all_sup_exc['inv_price_before_exc'] = all_sup_exc['price']*(1-all_sup_exc['discount']+all_sup_exc['lp_fee']*(1+all_sup_exc['invoice_fee']))
all_sup_exc['landed_cost'] = all_sup_exc['inv_price_before_exc'] * all_sup_exc['rate'] + all_sup_exc['shipping_cost']
all_sup_exc.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,landed_cost
0,9780262530873,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,4.99,0.38,3.47,1000,UK,0.0,0.0,1.231680,3.0938,7.280572
1,9780262530989,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,25.00,0.38,3.47,1000,UK,0.0,0.0,1.231680,15.5000,22.561044
2,9781681883052,N/A,HARPER COLLINS,N/A,ZOOKAL,AUD,24.50,0.00,3.47,150,AUS,0.0,0.0,0.662076,24.5000,19.690858
3,9780262531092,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,37.95,0.38,3.47,1000,UK,0.0,0.0,1.231680,23.5290,32.450205
4,9781681883212,N/A,HARPER COLLINS,N/A,ZOOKAL,AUD,21.00,0.00,3.47,150,AUS,0.0,0.0,0.662076,21.0000,17.373592


## 9. Restrictions

In [31]:
rest = pd.read_excel("../../../../Vendors/. Restrictions\All Restricted List.xlsx")
rest.columns = map(str.lower, rest.columns)
rest.rename(columns={'isbn13': 'isbn'}, inplace = True)
rest['isbn'] = rest['isbn'].astype(str)
for col in ['isbn', 'vendor']:
    rest[col] = rest.apply(lambda x: clean_up(x[col]), axis =1)

rest = rest[['isbn','vendor']]
rest.head()

,isbn,vendor
0,9780323567428,RISKY
1,9780500292105,RISKY
2,9781936523443,RISKY
3,9780134149530,RISKY
4,9780134205588,RISKY


In [32]:
# restricted to all suppliers
risky = rest[rest['vendor'] == 'RISKY']
all_sup_exc = all_sup_exc[~all_sup_exc['isbn'].isin(list(risky['isbn']))]

In [33]:
# restriction per supplier
all_rest = pd.merge(all_sup_exc, rest, how='left', left_on = ['isbn','supplier'], right_on = ['isbn', 'vendor'] )
all_rest[~(all_rest['vendor'].isna())].head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,landed_cost,vendor
3707,9781133308911,ETHICS,CENGAGE,JULIE C. VAN CAMP,ALEK,GBP,40.99,0.43,3.47,200,UK,0.005,0.0,1.23168,23.3643,32.247347,ALEK
25079,9781260187014,PACKAGE: ELEMENTARY STATISTICS: A STEP BY STEP...,MCGRAW,ALLAN G. BLUMAN,RAJAN,USD,276.65,0.48,0.50,10000,US,0.000,0.0,1.00000,143.8580,144.358000,RAJAN
42242,9781284107913,N/A,J&B,N/A,RAJAN,USD,44.95,0.38,0.50,10000,US,0.000,0.0,1.00000,27.8690,28.369000,RAJAN
42313,9781284126143,N/A,J&B,N/A,RAJAN,USD,79.95,0.38,0.50,10000,US,0.000,0.0,1.00000,49.5690,50.069000,RAJAN
43255,9780077831066,MOSAIC LEVEL 2 LISTENING/SPEAKING STUDENT BOOK...,MCGRAW,JAMI HANREDDY; ELIZABETH WHALLEY,ALEK,GBP,42.65,0.50,3.47,10000,UK,0.005,0.0,1.23168,21.3250,29.735582,ALEK


## 10. Getting all posible orders for cost_analysis

In [34]:
all_rest.drop(
    all_rest[
        (all_rest['supplier'] == all_rest['vendor'])
        ].index, inplace = True
)
all_rest.drop(columns=['vendor'], inplace = True)
all_rest = all_rest.sort_values(by=['landed_cost'])
all_rest.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,landed_cost
3135312,9780325011523,N/A,HEINEMANN,N/A,ALEK,USD,0.0,0.52,0.5,1000,US,0.005,0.0,1.0,0.0,0.5
3120656,9780325118703,N/A,HEINEMANN,N/A,ALEK,USD,0.0,0.52,0.5,1000,US,0.005,0.0,1.0,0.0,0.5
3120654,9780325118611,N/A,HEINEMANN,N/A,ALEK,USD,0.0,0.52,0.5,1000,US,0.005,0.0,1.0,0.0,0.5
3120652,9780325118574,N/A,HEINEMANN,N/A,ALEK,USD,0.0,0.52,0.5,1000,US,0.005,0.0,1.0,0.0,0.5
3120650,9780325118567,N/A,HEINEMANN,N/A,ALEK,USD,0.0,0.52,0.5,1000,US,0.005,0.0,1.0,0.0,0.5


## Extra: Getting suppliers from UK, CAN, US

In [40]:
UK_US_CAN = all_rest[
    (all_rest['origin'] == 'UK')
    | (all_rest['origin'] == 'US')
    | (all_rest['origin'] == 'CAN')
    ]

UK_US_CAN.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,landed_cost
3135312,9780325011523,N/A,HEINEMANN,N/A,ALEK,USD,0.0,0.52,0.5,1000,US,0.005,0.0,1.0,0.0,0.5
3120656,9780325118703,N/A,HEINEMANN,N/A,ALEK,USD,0.0,0.52,0.5,1000,US,0.005,0.0,1.0,0.0,0.5
3120654,9780325118611,N/A,HEINEMANN,N/A,ALEK,USD,0.0,0.52,0.5,1000,US,0.005,0.0,1.0,0.0,0.5
3120652,9780325118574,N/A,HEINEMANN,N/A,ALEK,USD,0.0,0.52,0.5,1000,US,0.005,0.0,1.0,0.0,0.5
3120650,9780325118567,N/A,HEINEMANN,N/A,ALEK,USD,0.0,0.52,0.5,1000,US,0.005,0.0,1.0,0.0,0.5


In [44]:
isbn_list = (
'9781305263727',
'9780323287531',
'9781305943278',
'9780323567022',
'9780323511407',
'9781469899817',
'9781684201372',
'9781260143393',
'9780323244855',
'9781259859618',
'9781259862908',
'9780323319751',
'9781118791356',
'9781337102278',
'9780071842617',
'9781462513925',
'9780357108291',
'9781337091992',
'9781284156959',
'9780323354820',
'9781259584732',
'9781285854182',
'9781284155204',
'9781543801637',
'9780323311274',
'9781259686207',
'9780323378390',
'9781284021028',
'9781337407816',
'9781464183959',
'9781464126130',
'9780323510806',
'9781305585126',
'9780073383095',
'9780199946563',
'9781451191554',
'9780133514995',
'9780323277884',
'9781337405713',
'9781337406420',
'9780323067768',
'9781259539060',
'9781543804300',
'9781118554135'
)

In [45]:
list_order = UK_US_CAN[UK_US_CAN['isbn'].isin(isbn_list)]

In [46]:
list_order.to_csv('retail/list_order.csv', index= False)